In [65]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline
from numpy.polynomial.polynomial import Polynomial
import plotly.graph_objects as go

In [84]:
df = pd.read_csv('energy_consumption.csv', parse_dates=[0])

In [85]:
df.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [88]:
df = df[:15]

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  15 non-null     datetime64[ns]
 1   PJME_MW   15 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 368.0 bytes


In [90]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = df['PJME_MW'],
                    x = df.Datetime,
                    line = dict(color = 'blue', width = 2),
                    name = 'Потребление энергии'))

fig.update_layout(title='Потребление энергии за 2002 - 2017г',
                    xaxis_title='Дата',
                    yaxis_title='Потребление энергии, МВт')
fig.show()

In [94]:
x_vals = np.arange(0, 10)
y_vals = df['PJME_MW'].values[:10]

x_new = np.arange(0, 15)

## Линейная интерполяция

In [95]:
linear_interp = interp1d(x_vals, y_vals, kind='linear', fill_value="extrapolate")
y_linear = linear_interp(x_new)

In [107]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = y_vals,
                    x = df['Datetime'][x_vals],
                    mode='markers',
                    name = 'Исходные данные'))

fig.add_trace(go.Scatter(y = y_linear,
                    x = df['Datetime'][x_new],
                    name = 'Потребление энергии',
                    mode='lines'))

fig.update_layout(title='Линейная интерполяция',
                    xaxis_title='Год',
                    yaxis_title='Потребление энергии, МВт')
fig.show()

## Кубический сплайн

In [98]:
cubic_spline = CubicSpline(x_vals, y_vals)
y_cubic = cubic_spline(x_new)

In [106]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = y_vals,
                    x = df['Datetime'][x_vals],
                    mode='markers',
                    name = 'Исходные данные'))

fig.add_trace(go.Scatter(y = y_cubic,
                    x = df['Datetime'][x_new],
                    name = 'Потребление энергии',
                    mode='lines'),
                    )

fig.update_layout(title='Кубический сплайн',
                    xaxis_title='Год',
                    yaxis_title='Потребление энергии, МВт')
fig.show()

## Интерполяция Лагранжа 

In [101]:
lagrange_interp = lagrange(x_vals, y_vals)
y_lagrange = lagrange_interp(x_new)

In [108]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = y_vals,
                    x = df['Datetime'][x_vals],
                    mode='markers',
                    name = 'Исходные данные'))

fig.add_trace(go.Scatter(y = y_lagrange,
                    x = df['Datetime'][x_new],
                    name = 'Потребление энергии',
                    mode='lines'))

fig.update_layout(title='Интерполяция Лагранжа',
                    xaxis_title='Год',
                    yaxis_title='Потребление энергии, МВт')
fig.show()

## Интерполяция Ньютона

In [46]:
def newton_coefficients(x_vals, y_vals):
    n = len(x_vals)
    coef = np.copy(y_vals)
    
    for i in range(1, n):
        coef[i:n] = (coef[i:n] - coef[i-1]) / (x_vals[i:n] - x_vals[i-1])
    
    return coef

def newton_interpolation(x_vals, y_vals, x):
    coef = newton_coefficients(x_vals, y_vals)
    n = len(x_vals)
    result = coef[-1]
    
    for i in range(n - 2, -1, -1):
        result = result * (x - x_vals[i]) + coef[i]
    
    return result

In [103]:
y_newton = [newton_interpolation(x_vals, y_vals, x) for x in x_new]

In [109]:
fig = go.Figure()

fig.add_trace(go.Scatter(y = y_vals,
                    x = df['Datetime'][x_vals],
                    mode='markers',
                    name = 'Исходные данные'))

fig.add_trace(go.Scatter(y = y_newton,
                    x = df['Datetime'][x_new],
                    name = 'Потребление энергии',
                    mode='lines'))

fig.update_layout(title='Интерполяция Ньютона',
                    xaxis_title='Год',
                    yaxis_title='Потребление энергии, МВт')
fig.show()